# Getting List of Most Discussed Topics on Github and Corresponding Repositories

### Importing Packages

In [162]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Defining Functions

In [163]:
# To get the DataFrame containing the datails of each topic
def get_topics_info(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text , 'html.parser')

    topic_names = []
    topic_desc = []
    topic_url = []

    topic_name_tags = soup.find_all('p' , class_ = 'f3 lh-condensed mb-0 mt-1 Link--primary')
    for topic_name_tag in topic_name_tags:
        topic_names.append(topic_name_tag.text)
        
    topic_desc_tags = soup.find_all('p' , class_ = 'f5 color-fg-muted mb-0 mt-1')
    for topic_desc_tag in topic_desc_tags:
        topic_desc.append(topic_desc_tag.text.strip())

    topic_url_tags = soup.find_all('a' , class_ = 'no-underline flex-grow-0')
    for topic_url_tag in topic_url_tags:
        topic_url.append('https://github.com'+topic_url_tag.get('href'))

    topic_dict = {
        "Topic Name" : topic_names,
        "Topic Description" : topic_desc,
        "Topic Url" : topic_url,
    }

    df = pd.DataFrame(topic_dict)
    return df

# To get the repositories related to given topic
def get_repo_info(url):    
    res = requests.get('https://github.com/topics/3d')
    soup = BeautifulSoup(res.text , 'html.parser')

    repo_names = []
    usernames = []
    url = []
    stars = []

    repo_tags = soup.find_all('h3' , class_ = 'f3 color-fg-muted text-normal lh-condensed')
    for repo_tag in repo_tags:
        a_tags = repo_tag.find_all('a')
        repo_names.append(a_tags[0].text.strip())
        usernames.append(a_tags[1].text.strip())
        url.append( 'https://github.com' + a_tags[1].get('href'))

    star_tags = soup.find_all('span' , class_ = 'Counter js-social-count')
    for star_tag in star_tags:
        star = int(float(star_tag.text.removesuffix('k')) * 1000)
        stars.append(star)

    repo_dict = {
        'Repo Name' : repo_names,
        'Username' : usernames,
        'Url' : url,
        'Stars' : stars
    }

    df = pd.DataFrame(repo_dict)
    return df

### Creating Files

In [164]:
url = 'https://github.com/topics'

df_topics = get_topics_info(url)
df_topics.to_csv('List of most discussed topics.csv')
len(df_topics)

30

In [165]:
for name,link in zip(df_topics['Topic Name'] , df_topics['Topic Url']):
    df_repo = get_repo_info(link)
    df_repo.to_csv(f'Topic_Repositories\\{name}.csv')
    len(df_repo)
